<h1 align="center">
    <img 
        src="./img/Microsoft-Logo.png" 
        width="400"/>
</h1>
<h1 align="center">
    <b>Practical Guide</b>
</h1>
<h4 align="center">
    for the creation of an AI Solution using an accelerator from the <a href="https://www.ds-toolkit.com/">Data Science Toolkit</a>
</h4>

# What to expect

* **Challenge 1:** *Create your own AI solution*
* **Challenge 2:** *Evaluate the quality of the AI solution*
* **Challenge 3:** *Create explanations to have insights on how to improve the quality of the AI solution*

# Challenge 1: *Create your own AI solution*

Here we are going to create a RAG based Copilot to answer questions about 6 sustainability reports of Microsoft, Apple, Amazon, Google, Meta and Netflix from 2022 or 2021. The documents are part of the  [Mini Esg Bench Dataset](https://llamahub.ai/l/llama_datasets/Mini%20ESG%20Bench%20Dataset?from=llama_datasets).

## Challenge 1 - Step 1:  Let's, first, install the required packages and libraries.

> This process will take around **1 minute** to complete. It is going to be done in a quite mode, only errors will be displayed if they occur. If you like to see what is going to be installed look at the [requirements.txt](./requirements.txt) file.

In summary two main tools will be installed that will be used in this notebook:

* **Llama Index**. Which will be used to download the dataset and to create the Semantic Index.
  > It is also possible to use **Azure AI Search** to create the semantic index, but since it is going to be an small index, just to simplify its creation in-memory, we are going to use Llama Index in this example.
* **RAGAS**. Ragas is going to be used to calculate the quality metrics for the Copilot that we are going to create.

In [1]:
%%time
!pip install -q -q -r requirements.txt

CPU times: user 21.9 ms, sys: 16.1 ms, total: 38 ms
Wall time: 1.6 s


## Challenge 1 - Step 2: *Let's import the libraries to be used in this notebook*

In [2]:
import os
import pandas as pd

# To create the RAG based copilot
from llama_index.core.llama_dataset import download_llama_dataset, LabelledRagDataset
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, Settings
from ragas.metrics import (
    Faithfulness,
    ContextPrecision,
    ContextRecall
)

# To calculate the Generative AI quality metrics
from ragas.llms import LlamaIndexLLMWrapper
from ragas.embeddings import LlamaIndexEmbeddingsWrapper
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas.evaluation import evaluate
from ragas.run_config import RunConfig

# To create a simple PDF visualization tool
import pymupdf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output


## Challenge 1 - Step 3: *Let's download the documents and test questions*

> This step takes around **1 minute** to complete depending on the internet connection.

In [3]:
rag_dataset, documents = download_llama_dataset(
    llama_dataset_class="MiniEsgBenchDataset", 
    download_dir="./data",
    show_progress=True
)

Loading files: 100%|██████████| 6/6 [00:44<00:00,  7.48s/file]


### Little tool to visualize the reports just downloaded

The following cell creates a simple tool to quickly visualize the PDF files just downloaded. If you like to take a look at the downloaded reports manually, just navigate to the `data/source_files` folder.

In [4]:
data_source_path = "./data/source_files"

# List of pdf files just downloaded
pdf_files = [os.path.join(data_source_path, file_name) for file_name in os.listdir(data_source_path)]

# Function to render a specific page of a PDF
def render_pdf_page(pdf_path, page_number=0):
    # Open the PDF file
    pdf_document = pymupdf.open(pdf_path)
    
    # Ensure the page number is valid
    if page_number < 0 or page_number >= len(pdf_document):
        raise ValueError("Invalid page number.")
    
    # Get the page and render it as an image
    page = pdf_document[page_number]
    pix = page.get_pixmap()
    pdf_document.close()
    
    # Display the image using Matplotlib
    plt.figure(figsize=(10, 8))
    plt.imshow(pix.pil_image())
    plt.axis("off")
    plt.show()

# Function to update the displayed page
def update_page(step):
    global current_page
    pdf_document = pymupdf.open(dropdown.value)
    total_pages = len(pdf_document)
    pdf_document.close()
    
    # Update the current page index
    current_page += step
    if current_page < 0:
        current_page = 0
    elif current_page >= total_pages:
        current_page = total_pages - 1
    
    with output:
        output.clear_output()
        render_pdf_page(dropdown.value, current_page)

# Function to reset the viewer when a new PDF is selected
def reset_viewer(change):
    global current_page
    current_page = 0  # Reset to the first page
    with output:
        output.clear_output()
        render_pdf_page(dropdown.value, current_page)

# Create widgets
dropdown = widgets.Dropdown(
    options=pdf_files,
    description="Select PDF:",
    style={"description_width": "initial"}
)

prev_button = widgets.Button(description="Previous Page")
next_button = widgets.Button(description="Next Page")
output = widgets.Output()

# Attach event listeners
prev_button.on_click(lambda _: update_page(-1))
next_button.on_click(lambda _: update_page(1))
dropdown.observe(reset_viewer, names="value")

# Initial display
reset_viewer(None)

# Display widgets and output
display(widgets.VBox([dropdown, widgets.HBox([prev_button, next_button]), output]))

## Challenge 1 - Step 4: *Let's create the semantic index*

> This process can take up to **3 minutes** to complete
> 
> **TODO:** Explain how the Semantic index works

In [5]:
embed_model = AzureOpenAIEmbedding(
    model='text-embedding-3-small', # Update with the embeddings deployment name
    api_key=os.environ['OPENAI_API_KEY'],
    api_version=os.environ['OPENAI_API_VERSION'],
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
)

Settings.embed_model = embed_model

index = VectorStoreIndex.from_documents(
    documents=documents,
    show_progress=True
)


Parsing nodes:   0%|          | 0/455 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/512 [00:00<?, ?it/s]

## Challenge 1 - Step 5: *Let's create the copilot*

> **TODO:** Explain how the copilot is created 

In [7]:
llm = AzureOpenAI(
    engine="gpt-4o", # Update with the language model deployment name 
    model="gpt-4o", # Update with the language model name
    temperature=0.0,
    api_key=os.environ['OPENAI_API_KEY'],
    api_version=os.environ['OPENAI_API_VERSION'],
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
)

Settings.llm = llm

query_engine = index.as_query_engine() # this is the copilot

### Let's play with the copilot just created

In the test dataset, we have not only downloaded the source data, but also 50 example questions to ask to the copilot. The following are some examples of the questions:

In [8]:
num_samples = 3
samples = rag_dataset.to_pandas().loc[:num_samples-1, 'query'].values

display(Markdown("\n".join([ "* " + sample for sample in samples])))

* Can you provide for me the three highlights for the GHG emissions section of the Advancing Carbon-Free Energy Performance Highlights?
* What percentage of waste from Google's offices globally were diverted away from landfills in 2021?
* Can you present me with the performance highlights for Empowering Users With Technology?

In [9]:
# Function to get a response
def get_response(user_input):
    return query_engine.query(user_input).response

# Interactive chatbot function
def chatbot():
    output = widgets.Output()
    text_box = widgets.Text(
        placeholder="Type your message here",
        description="You:",
        style={'description_width': 'initial'}
    )
    
    # Callback function for observing changes
    def on_value_change(change):
        if change['type'] == 'change' and change['name'] == 'value' and change['new']:  # Ensure it's a valid change
            with output:
                clear_output(wait=True)
                user_message = change['new']
                bot_response = get_response(user_message)
                print(f"You: {user_message}")
                print(f"\nESG Bot: {bot_response}")
            text_box.value = ""  # Clear the input box after submission

    # Attach observer to the text box
    text_box.observe(on_value_change, names='value')
    
    display(text_box, output)

# Run the chatbot
chatbot()

Text(value='', description='You:', placeholder='Type your message here', style=TextStyle(description_width='in…

Output()

# Challenge 2: *Evaluation of the quality of our new Copilot*

## Challenge 2 - Step 1: *Let's take a look at the test dataset*

First, let's see an example of each instance of the test dataset that we downloaded together with the PDF files.

In [10]:
instance_idx = 10

def create_instance_md(k, v):
    md_str = f"**{k}:**\n\n"
    if k == "reference_contexts":
        return md_str + "\n".join([f"* {c}\n" for c in v])
    return md_str + f"{v}"

display(Markdown("\n\n".join([create_instance_md(k, v) for k,v in rag_dataset.to_pandas().iloc[instance_idx].items()])))

**query:**

Can you provide the Actions taken by Amazon for achieving their Carbon goals?

**reference_contexts:**

* Actions
90% Electricity consumed by Amazon attributable to renewable energy sources, up from 85% in 2021
29% Reduction in Scope 2 emissions from purchased electricity
7 New companies invested in through The Climate Pledge Fund to develop breakthrough low-carbon technologies to help transform the industry
145M Packages delivered by EVs in the U.S. and Europe
111 New signatories of The Climate Pledge


**reference_answer:**

Sure, here they are:
Actions
90% Electricity consumed by Amazon attributable to renewable energy sources, up from 85% in 2021
29% Reduction in Scope 2 emissions from purchased electricity
7 New companies invested in through The Climate Pledge Fund to develop breakthrough low-carbon technologies to help transform the industry
145M Packages delivered by EVs in the U.S. and Europe
111 New signatories of The Climate Pledge

**reference_answer_by:**

human

**query_by:**

human

### Full dataset
In total, the test dataset have 50 instances like the one detailed before, Let's take a look at the full dataset

In [11]:
rag_dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,Can you provide for me the three highlights fo...,[GHG emissions\n65%\ncumulative GHG\nemissions...,"Sure, they are: \n1. 65% cumulative GHG emissi...",human,human
1,What percentage of waste from Google's offices...,"[64%\nlandfill diversion\nIn 2021, we reached ...",Sixty-four percent.,human,human
2,Can you present me with the performance highli...,[EMPOWERING USERS WITH TECHNOLOGY\nProducts\nT...,Sure! The Performance Highlights for Empowerin...,human,human
3,What was the listed key achievement regarding ...,[We’ve been a leader on sustainability and cli...,"In 2017, Google became the first major company...",human,human
4,Did Google reach its intended Waste target und...,[Target: Achieve UL 2799 Zero Waste to Landfil...,"No, this target has not been met in 2021. Howe...",human,human
5,How many EV charging locations were there on G...,"[200,000\nEV charging locations\non Google Map...",200000,human,human
6,On what page of the report can I find the perf...,[EMPOWERING USERS WITH TECHNOLOGY\nProducts\nT...,The performance highlights for Empowering User...,human,human
7,Can you please provide for me the glossary of ...,[Glossary\nCFE: carbon-free energyCO2e: carbon...,"Sure, here is the glossary:\nGlossary\nCFE: ca...",human,human
8,On what page can I find details about Amazons ...,[Contents\nIntroduction\n2 About Amazon\n3 Ope...,You can find information on driving climate so...,human,human
9,"For the listed Renewable Energy goals, by when...",[Renewable Energy\nGoal: Power our operations ...,Amazon set the goal of becoming powered by 100...,human,human


In [12]:
sample_size = 5
sub_dataset = LabelledRagDataset(examples=rag_dataset.examples[:sample_size])
sub_dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,Can you provide for me the three highlights fo...,[GHG emissions\n65%\ncumulative GHG\nemissions...,"Sure, they are: \n1. 65% cumulative GHG emissi...",human,human
1,What percentage of waste from Google's offices...,"[64%\nlandfill diversion\nIn 2021, we reached ...",Sixty-four percent.,human,human
2,Can you present me with the performance highli...,[EMPOWERING USERS WITH TECHNOLOGY\nProducts\nT...,Sure! The Performance Highlights for Empowerin...,human,human
3,What was the listed key achievement regarding ...,[We’ve been a leader on sustainability and cli...,"In 2017, Google became the first major company...",human,human
4,Did Google reach its intended Waste target und...,[Target: Achieve UL 2799 Zero Waste to Landfil...,"No, this target has not been met in 2021. Howe...",human,human


In [13]:
%%time
predictions = sub_dataset.make_predictions_with(
    predictor = query_engine,
    show_progress = True
)

100%|██████████| 5/5 [01:24<00:00, 16.93s/it]

CPU times: user 482 ms, sys: 15.1 ms, total: 497 ms
Wall time: 1min 24s


In [14]:
list_of_samples = []

for idx in range(len(sub_dataset.examples)):
    list_of_samples.append(
        SingleTurnSample (
            user_input = sub_dataset.examples[idx].query,
            reference = sub_dataset.examples[idx].reference_answer,
            response = predictions.predictions[idx].response,
            retrieved_contexts = predictions.predictions[idx].contexts
        )
    )

ragas_evaluation_dataset = EvaluationDataset(list_of_samples)
ragas_evaluation_dataset.to_pandas()

,user_input,retrieved_contexts,response,reference
0,Can you provide for me the three highlights fo...,"[31. In 2018, to align with industry best prac...",The three highlights for the GHG emissions sec...,"Sure, they are: \n1. 65% cumulative GHG emissi..."
1,What percentage of waste from Google's offices...,[Performance highlights\nThe following section...,The percentage of waste from Google's global d...,Sixty-four percent.
2,Can you present me with the performance highli...,"[Education\nFor more than 40 years, we’ve work...",The performance highlights for empowering user...,Sure! The Performance Highlights for Empowerin...
3,What was the listed key achievement regarding ...,[Our approach\nWe believe that every business ...,There is no listed key achievement for Google ...,"In 2017, Google became the first major company..."
4,Did Google reach its intended Waste target und...,[BUILDING BETTER DEVICES AND SERVICES\nTarget ...,"Yes, in 2021, Google achieved the UL 2799 Zero...","No, this target has not been met in 2021. Howe..."


In [15]:
evaluator_llm = LlamaIndexLLMWrapper(llm)
evaluator_embeddings = LlamaIndexEmbeddingsWrapper(embed_model)

In [16]:
%%time

metrics = [
    Faithfulness(llm=evaluator_llm),
    ContextPrecision(llm=evaluator_llm),
    ContextRecall(llm=evaluator_llm)
]
ragas_evaluation_result = evaluate(
    dataset=ragas_evaluation_dataset,
    metrics=metrics,
    llm=evaluator_llm,
    embeddings=evaluator_embeddings,
    run_config=RunConfig(timeout=1800, max_wait=180, max_retries=20),
    show_progress=True,
    batch_size=5
)

Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

Batch 1/3:   0%|          | 0/5 [00:00<?, ?it/s]

Retrying llama_index.llms.openai.base.OpenAI._achat in 1.0 seconds as it raised RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-08-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 59 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}.


CPU times: user 734 ms, sys: 59.2 ms, total: 793 ms
Wall time: 6min 5s


In [17]:
df_ragas_result = ragas_evaluation_result.to_pandas()
df_ragas_result

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision,context_recall
0,Can you provide for me the three highlights fo...,"[31. In 2018, to align with industry best prac...",The three highlights for the GHG emissions sec...,"Sure, they are: \n1. 65% cumulative GHG emissi...",1.000000,0.0,0.0
1,What percentage of waste from Google's offices...,[Performance highlights\nThe following section...,The percentage of waste from Google's global d...,Sixty-four percent.,1.000000,0.0,0.0
2,Can you present me with the performance highli...,"[Education\nFor more than 40 years, we’ve work...",The performance highlights for empowering user...,Sure! The Performance Highlights for Empowerin...,1.000000,0.0,0.0
3,What was the listed key achievement regarding ...,[Our approach\nWe believe that every business ...,There is no listed key achievement for Google ...,"In 2017, Google became the first major company...",1.000000,1.0,1.0
4,Did Google reach its intended Waste target und...,[BUILDING BETTER DEVICES AND SERVICES\nTarget ...,"Yes, in 2021, Google achieved the UL 2799 Zero...","No, this target has not been met in 2021. Howe...",0.666667,1.0,1.0


In [18]:
df_test_dataset = pd.read_json('./test-dataset.json', orient='records')
df_test_dataset

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision,context_recall
0,"In the essay, the author mentions his early ex...",[What I Worked On\n\nFebruary 2021\n\nBefore c...,The first computer the author used for program...,The first computer the author used for program...,1.000000,1.0,1.000000
1,The author switched his major from philosophy ...,[I couldn't have put this into words when I wa...,The author developed an interest in AI due to ...,The two specific influences that led the autho...,0.900000,1.0,1.000000
2,"In the essay, the author discusses his initial...",[I couldn't have put this into words when I wa...,The author was initially drawn to AI by two ma...,The two main influences that initially drew th...,1.000000,1.0,1.000000
3,The author mentions his shift of interest towa...,[I couldn't have put this into words when I wa...,The author shifted his interest towards Lisp b...,The author shifted his interest towards Lisp a...,0.900000,1.0,1.000000
4,"In the essay, the author mentions his interest...",[So I looked around to see what I could salvag...,"During his time in grad school, the author att...","The author in the essay is Paul Graham, who wa...",0.846154,1.0,1.000000
5,The author discusses his decision to write a b...,[I couldn't have put this into words when I wa...,The author decided to write a book on Lisp hac...,The author decided to write a book on Lisp hac...,0.500000,1.0,1.000000
6,"In the essay, the author mentions a quick deci...",[So I looked around to see what I could salvag...,The author made a quick decision to attempt to...,The author decided to attempt writing his diss...,1.000000,1.0,1.000000
7,The author describes the atmosphere and practi...,"[I didn't want to drop out of grad school, but...",The author describes the atmosphere at the Acc...,"According to the author's account, the student...",1.000000,1.0,1.000000
8,"In the essay, the author discusses his experie...","[We actually had one of those little stoves, f...",The author describes painting still lives as d...,"In the essay, the author explains that paintin...",0.923077,1.0,1.000000
9,The author shares his work experience at a com...,"[We actually had one of those little stoves, f...",Interleaf had added a unique feature to their ...,"Interleaf, the company where the author worked...",0.812500,1.0,0.857143
